<a href="https://colab.research.google.com/github/YiChihHuang/shortreportcreator/blob/main/shortreportcreater_0812.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = '輸入你的API KEY'
os.environ['TAVILY_API_KEY'] = '輸入你的API KEY' # Get a free key here: https://app.tavily.com

In [ ]:
!pip install -U gpt-researcher nest_asyncio

In [ ]:
!pip install openai python-dotenv tavily-python python-docx

In [ ]:
import os
import re
from openai import OpenAI
from tavily import TavilyClient
import json
from docx import Document
from docx.shared import Pt
from docx.enum.style import WD_STYLE_TYPE
from google.colab import files
from dotenv import load_dotenv

# 加載環境變量
load_dotenv()

# 設置API密鑰
openai_api_key = os.getenv("OPENAI_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")

# 如果環境變量未設置，提示用戶輸入
if not openai_api_key:
    openai_api_key = input("請輸入您的OpenAI API密鑰: ")
    os.environ["OPENAI_API_KEY"] = openai_api_key

if not tavily_api_key:
    tavily_api_key = input("請輸入您的Tavily API密鑰: ")
    os.environ["TAVILY_API_KEY"] = tavily_api_key

# 初始化客戶端
openai_client = OpenAI(api_key=openai_api_key)
tavily_client = TavilyClient(api_key=tavily_api_key)

def search_info(query, sources):
    """使用Tavily搜索相關信息"""
    try:
        results = []
        for source in sources:
            if source.lower() == "all":
                response = tavily_client.search(query=query, search_depth="advanced", max_results=10)
            else:
                response = tavily_client.search(query=f"{query} site:{source}", search_depth="advanced", max_results=2)
            results.extend(response.get('results', []))
        print(f"Tavily API 返回 {len(results)} 個結果")
        return results
    except Exception as e:
        print(f"搜索過程中發生錯誤: {str(e)}")
        return []

def generate_report(topic, search_results):
    """使用OpenAI生成研究報告"""
    try:
        sources = [result.get('url', '') for result in search_results if isinstance(result, dict)]
        sources_text = "\n".join(f"{i+1}. {url}" for i, url in enumerate(sources))

        content_text = "\n".join(result.get('content', '') for result in search_results if isinstance(result, dict))

        prompt = f"""
        根據以下資訊,使用繁體中文撰寫一份關於"{topic}"的研究報告:

        {content_text}

        請確保報告內容:
        1. 使用正式的繁體中文
        2. 包含引言、主要研究發現、討論和結論部分
        3. 在每個重要論點後添加來源引用,採用APA格式的括號內文引用,使用(作者, 年份)標記，例如(Oxford Economics, 2012)
        4. 僅使用提供的資料來源進行引用，不要創造或假設其他來源
        5. 在每個章節最後提供該章節的"參考文獻"部分,列出所有引用的來源
        6. 不需要在報告末尾添加參考文獻列表，這將在後處理中自動添加


        以下是可以引用的來源網址列表（請只使用這些來源），請確保引用中的年份與作者名正確，以作者年代方式排序，中文文獻在前、英文文獻在後，不採編號方式排列，中文以作者姓氏筆畫（由少至多）排序，英文以作者姓氏字母（由A到Z）排序並在文末提供完整的參考文獻格式:
        {sources_text}
        """

        response = openai_client.chat.completions.create(
            model="gpt-4-0125-preview",  # 更新為正確的模型名稱
            messages=[
                {"role": "system", "content": "你是一個專門撰寫繁體中文研究報告的AI助手。你的任務是基於提供的資訊撰寫全面、深入且結構良好的研究報告。"},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=4000
        )

        return response.choices[0].message.content  # 修正：返回生成的報告內容

    except Exception as e:
        print(f"生成報告過程中發生錯誤: {str(e)}")
        return f"生成報告時發生錯誤: {str(e)}"

def create_word_document(content, filename):
    """創建包含研究報告內容的Word文檔"""
    try:
        doc = Document()

        # 設置標題樣式
        title_style = doc.styles.add_style('TitleStyle', WD_STYLE_TYPE.PARAGRAPH)
        title_font = title_style.font
        title_font.name = '標楷體'
        title_font.size = Pt(16)
        title_font.bold = True

        # 設置正文樣式
        body_style = doc.styles.add_style('BodyStyle', WD_STYLE_TYPE.PARAGRAPH)
        body_font = body_style.font
        body_font.name = '標楷體'
        body_font.size = Pt(12)

        # 添加標題
        doc.add_paragraph("研究報告", style='TitleStyle')

        # 添加正文內容
        paragraphs = content.split('\n')
        for para in paragraphs:
            doc.add_paragraph(para, style='BodyStyle')

        # 保存文檔
        doc.save(filename)
        return filename
    except Exception as e:
        print(f"創建Word文檔時發生錯誤: {str(e)}")
        return None

def research_report_agent(topic, sources):
    """主要的agent函數"""
    search_results = search_info(topic, sources)
    if not search_results:
        return "無法獲取搜索結果", None

    report = generate_report(topic, search_results)

    # 創建Word文檔
    filename = f"{topic}_研究報告.docx"
    word_file = create_word_document(report, filename)

    return report, word_file

# 主程序
if __name__ == "__main__":
    topic = input("請輸入研究主題: ")
    sources_input = input("請輸入資訊來源 (輸入 'all' 搜索所有來源，或輸入至多5個指定url，用逗號分隔): ")
    sources = [s.strip() for s in sources_input.split(',') if s.strip()]
    if len(sources) > 5:
        print("警告：您輸入的來源超過5個，只會使用前5個來源。")
        sources = sources[:5]

    report, word_file = research_report_agent(topic, sources)
    if word_file:
        print(f"研究報告已生成並保存至 {word_file}")
        files.download(word_file)  # 自動下載生成的Word文檔
    else:
        print("無法創建Word文檔")
    print("\n報告內容預覽:")
    print(report[:500] + "...") # 只顯示前500個字符作為預覽

請輸入研究主題: 2024年台灣博士人才培育方案研析
請輸入資訊來源 (輸入 'all' 搜索所有來源，或輸入至多5個指定url，用逗號分隔): https://www.heeact.edu.tw, https://www.edu.tw, https://www.worldjournal.com, https://www.thenewslens.com, https://epaper.naer.edu.tw
Tavily API 返回 10 個結果
研究報告已生成並保存至 2024年台灣博士人才培育方案研析_研究報告.docx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


報告內容預覽:
# 2024年台灣博士人才培育方案研析

## 引言

在全球化與科技快速發展的當代，博士級研發人才的培育成為各國競逐的焦點。台灣作為一個科技進步的國家，其高等教育及產業發展對於博士級人才的需求日益增加。面對此一挑戰，教育部及相關機構推出多項策略與計畫，旨在培育符合產業需求的博士級研發人才，以縮小學用落差並促進產學合作。

## 主要研究發現

### 產學合作培育博士級研發人才計畫

教育部自103年起推動的產博計畫，目的在於降低學用落差，提高企業對博士級人才的進用意願（顏瑄慧，無年份）。此計畫通過大學與產業共同合作，以實際的產業需求為導向，培養具備實務研發能力的博士級人才。

### 人才永續行動聯盟

台灣人才永續行動聯盟與超過百家企業和學校共同推動人才的發展與培育，展現對「意義與價值」的承諾，透過研究、推廣工程及科技教育，服務社會，促進國家經濟發展（顏瑄慧，無年份）。

### 前瞻顯示科技與跨域培育人才計畫

教育部推動的「前瞻顯示科技與跨領域人才培育計畫」旨在鼓勵大學培育顯示科技人才，以接軌投入產業發展（資訊及科技教育司，無年份）。這項計畫展現了政府在跨領域人才培育方面的積...
